Sucht alle Einträge mit RTG "event" über die API, 
gibt in Excel aus:
DOIs
Publication year (in Confident = Jahr der Konferenz, wie machen es andere?), 
Titel (Freitext) (benötigt v.a. für Zenodo, weil deren Einträge sehr oft ohne Resource Type - s. unten, weil Spam)
ResourceType (Freitext)

issue 1 (erledigt):
Bei Suche nach allen bricht die Suche mit page number nach 10k Einträgen ab. 
Laut API-Suche im Browser gibt es > 78k Event-Einträge, größtes Repo allein hat > 50k
Lösung siehe
https://support.datacite.org/docs/pagination
Cursor kann nicht manuell inkrementiert werden, deswegen zurück-umschreiben auf Abruf der nächsten Seite aus json > links > next

issue 2 (erledigt):
Absurd viele zenodo-DOIs ohne RT-Eintrag und Spam als Titel, deswegen Zenodo aus Anfrage ausnehmen

In [42]:
import requests
import pandas as pd

# Define the API URL
URL = "https://api.datacite.org/dois"

# Define the query parameters
params = {
    "resource-type-id": "Event",
    #exclude Zenodo from results, due to massive spam problem. Or query for specific repository
    "query": "NOT publisher:Zenodo",
    # Use cursor-based pagination to retrieve more than 10 000 DOI-MD-sets
    "page[cursor]": 1,
    "page[size]": 10
}

# Initialize lists to store data
dois = []
publication_years = []
titles = []
rtype = []

# Initial request to API 
response = requests.get(URL, params=params)

# Function to fetch data from API and append to lists, and return URL of next page of results

def fetch_data():
    global response
    antwort = response.json()
    for item in antwort["data"]:
        attributes = item["attributes"]
        
        dois.append(attributes["doi"])
        publication_years.append(attributes["publicationYear"])
        titles_item = []
        for i in range(len(attributes["titles"])):
            titles_item.append(attributes["titles"][i]["title"])
        titles.append(titles_item)
        rtype.append(attributes["types"].get("resourceType", "none given"))
        
    next_url = antwort["links"].get("next")
    return next_url
    

# Fetch data and request next page of results 

# "while True"-loop should be stopped by (falsey) "None" returned by default by the .get-method
# If an error is returned anyway,try antwort["links"].get("next", False) above

#while True:
i = 0
while i < 4:
    next_url = fetch_data()
    response = requests.get(next_url)
    #print(response.url)
    i += 1
    

# Create a DataFrame from the extracted data


# Show DataFrame untruncated
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)
pd.set_option('display.width',1000)


df = pd.DataFrame({
    "DOI": dois,
    "Publication Year": publication_years,
    "Title" : titles,
    "Resource Type": rtype
})

# Generate a statistic of events per publication year
event_counts = df["Publication Year"].value_counts().sort_index()

# Print the statistic
print("Number of Events registered: ", len(dois))
print("Statistic of events per publication year:")
print(event_counts)


Number of Events registered:  40
Statistic of events per publication year:
Publication Year
2005    40
Name: count, dtype: int64


In [43]:
#df.to_excel("EventDOIs_RT_RTG.xlsx")

In [44]:
print(df)

                     DOI  Publication Year                                              Title            Resource Type
0   10.4122/1.1000001019              2005  [A Decision Support System for Urban Groundwat...  Conference presentation
1   10.4122/1.1000001021              2005  [New Act Legislated in Japan for Well-balanced...  Conference presentation
2   10.4122/1.1000001023              2005  [Computer Aided Rehabilitation of Water Networ...  Conference presentation
3   10.4122/1.1000001025              2005  [Construction and testing of a CSO treating co...  Conference presentation
4   10.4122/1.1000001027              2005  [The social impacts of stormwater management t...  Conference presentation
5   10.4122/1.1000001029              2005  [Stormwater quality as described in the Nation...  Conference presentation
6   10.4122/1.1000001031              2005  [Integrating on-site water management practice...  Conference presentation
7   10.4122/1.1000001033              2005  [Int